In [19]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [5]:
#Read a pickle file and load the data
import pickle
with open('final_df.pkl', 'rb') as file:
    final_df = pickle.load(file)

In [8]:
df = final_df
df['diagnosis']=df['diagnosis'].astype('int')
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302.0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517.0,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903.0,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301.0,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402.0,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424.0,1,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682.0,1,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954.0,1,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241.0,1,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    float64
 1   diagnosis                569 non-null    int64  
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [10]:
y=df['diagnosis']
X = df.drop(columns=['diagnosis'])

In [ ]:
# Hyperparameters

In [ ]:
## Multivariable Analysis

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

In [12]:
### Summarization and Selection of Variables

In [17]:
# Fit models and determine if a feature is selected (1) or not (0)
lasso = Lasso(alpha=0.01).fit(X, y)
lasso_selected = (np.abs(lasso.coef_) > 0).astype(int)

svm = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
svm_selected = (np.abs(svm.coef_[0]) > 0).astype(int)

gb = GradientBoostingClassifier().fit(X, y)
gb_selected = (gb.feature_importances_ > 0).astype(int)

rf = RandomForestClassifier().fit(X, y)
rf_selected = (rf.feature_importances_ > 0).astype(int)

# Create a DataFrame to store results
selection_df = pd.DataFrame({
    'Feature': X.columns,
    'Lasso': lasso_selected,
    'SVM': svm_selected,
    'GradientBoost': gb_selected,
    'RandomForest': rf_selected
})

# Sum the number of selections for each feature
selection_df['Sum'] = selection_df[['Lasso', 'SVM', 'GradientBoost', 'RandomForest']].sum(axis=1)

# Output the results
print(selection_df)

                    Feature  Lasso  SVM  GradientBoost  RandomForest  Sum
0                        id      1    1              1             1    4
1               radius_mean      0    1              1             1    3
2              texture_mean      1    1              1             1    4
3            perimeter_mean      0    1              1             1    3
4                 area_mean      1    1              1             1    4
5           smoothness_mean      0    0              1             1    2
6          compactness_mean      0    0              1             1    2
7            concavity_mean      0    0              1             1    2
8       concave points_mean      0    0              1             1    2
9             symmetry_mean      0    0              1             1    2
10   fractal_dimension_mean      0    0              1             1    2
11                radius_se      0    0              1             1    2
12               texture_se      0    

In [ ]:
# Creating DataFrame with most valuable variables

In [ ]:
# Selected variables - recommended by 3 or more models

In [21]:
 #Selecting variables with a sum of selections >= 3
final_var = selection_df[selection_df['Sum'] >= 3]['Feature'].tolist()
df_model = df[final_var].copy()
df_model['diagnosis'] = df['diagnosis'].copy()

# Output the result to verify
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               569 non-null    float64
 1   radius_mean      569 non-null    float64
 2   texture_mean     569 non-null    float64
 3   perimeter_mean   569 non-null    float64
 4   area_mean        569 non-null    float64
 5   area_se          569 non-null    float64
 6   radius_worst     569 non-null    float64
 7   texture_worst    569 non-null    float64
 8   perimeter_worst  569 non-null    float64
 9   area_worst       569 non-null    float64
 10  concavity_worst  569 non-null    float64
 11  diagnosis        569 non-null    int64  
dtypes: float64(11), int64(1)
memory usage: 53.5 KB


In [22]:
df_model.to_csv('df_model.csv')

In [ ]:
# Feature Selection Strategy

In [ ]:
## Univariable Analysis 
# Anova, t-test, Chi- square

In [ ]:
# Using Anova

In [23]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.datasets import load_iris
import pandas as pd
import seaborn as sns

In [25]:
# Load data
#Read a pickle file and load the data
import pickle
with open('final_df.pkl', 'rb') as file:
    final_df = pickle.load(file)

df = final_df
df['diagnosis']=df['diagnosis'].astype('int')
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302.0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517.0,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903.0,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301.0,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402.0,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424.0,1,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682.0,1,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954.0,1,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241.0,1,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [26]:
y=df['diagnosis']
X = df.drop(columns=['diagnosis'])

In [27]:
# Calculate F-values and p-values
f_values, p_values = f_classif(X, y)

# Create a DataFrame to display F-values and p-values for each feature
feature_scores = pd.DataFrame({'Feature': X.columns, 'F-Value': f_values, 'P-Value': p_values})

# Print the DataFrame
print(feature_scores)

                    Feature     F-Value        P-Value
0                        id    0.898150   3.436819e-01
1               radius_mean  646.981021   8.465941e-96
2              texture_mean  118.096059   4.058636e-25
3            perimeter_mean  697.235272  8.436251e-101
4                 area_mean  573.060747   4.734564e-88
5           smoothness_mean   83.651123   1.051850e-18
6          compactness_mean  313.233079   3.938263e-56
7            concavity_mean  533.793126   9.966556e-84
8       concave points_mean  861.676020  7.101150e-116
9             symmetry_mean   69.527444   5.733384e-16
10   fractal_dimension_mean    0.093459   7.599368e-01
11                radius_se  268.840327   9.738949e-50
12               texture_se    0.039095   8.433320e-01
13             perimeter_se  253.897392   1.651905e-47
14                  area_se  243.651586   5.895521e-46
15            smoothness_se    2.557968   1.102966e-01
16           compactness_se   53.247339   9.975995e-13
17        

In [29]:
# Select the top 2 features based on F-value
top_features = feature_scores.nlargest(2, 'F-Value')

# Print the 2 best features
print(top_features)

                 Feature     F-Value        P-Value
28  concave points_worst  964.385393  1.969100e-124
23       perimeter_worst  897.944219  5.771397e-119
